## Scaling Factors Calculation for H $\to$ WW Analysis

# Algorithm: 

## Overview
This algorithm calculates MC normalization scale factors, K-factor corrections, and signal strength measurements for Higgs to WW analysis across multiple jet categories.

---

## INPUT

1. **Luminosity** ($\mathcal{L}$) for run period -- Check the luminosity from official resource
2. **Sample information** for each MC sample $i$:
   - Cross-section: $\sigma_i$ (in pb)  -- use the [latinos github](https://github.com/latinos/LatinoAnalysis/blob/master/NanoGardener/python/framework/samples/samplesCrossSections2018.py) for this
   - Sum of generator weights: $\sum w_{\text{gen},i}$
   - Sample type: signal/background/top
3. **Raw event yields** for each sample in each category (0Z, 1Z, 2Z)
4. **Observed data events** per category

---

## PART 1: Calculate MC Normalization Scale Factors

For each MC sample $i$:

### Step 1.1: Convert luminosity from fb$^{-1}$ to pb$^{-1}$

$$
\mathcal{L}_{\text{pb}} = \mathcal{L} \times 1000
$$

### Step 1.2: Calculate scale factor

$$
\text{SF}_i = \frac{\sigma_i \times \mathcal{L}_{\text{pb}}}{\sum w_{\text{gen},i}}
$$

### Step 1.3: Store scale factor

Store $\text{SF}_i$ for sample $i$

---

## PART 2: Apply Scale Factors to Raw Event Yields

For each category $c \in \{0Z, 1Z, 2Z\}$:

### Step 2.1: Get raw event count

$$
N_{\text{raw},i,c} = \text{raw count for sample } i \text{ in category } c
$$

### Step 2.2: Apply scale factor

$$
N_{\text{scaled},i,c} = N_{\text{raw},i,c} \times \text{SF}_i
$$

### Step 2.3: Store scaled yield

Store $N_{\text{scaled},i,c}$

---

## PART 3: Calculate K-factor for Top Background Correction

For each category $c$:

### Step 3.1: Sum all top MC yields

$$
N_{\text{top},c} = \sum_{i \in \text{top samples}} N_{\text{scaled},i,c}
$$

### Step 3.2: Sum all non-top MC yields

$$
N_{\text{other},c} = \sum_{i \notin \text{top samples}} N_{\text{scaled},i,c}
$$

### Step 3.3: Get observed data

$$
N_{\text{data},c} = \text{observed events in category } c
$$

### Step 3.4: Calculate K-factor

$$
K_c = \frac{N_{\text{data},c} - N_{\text{other},c}}{N_{\text{top},c}}
$$

### Step 3.5: Store K-factor

Store $K_c$ for category $c$

---

## PART 4: Apply K-factor to Top Samples

For each category $c$:

For each top sample $i$:

### Step 4.1: Apply K-factor correction

$$
N_{\text{final},i,c} = N_{\text{scaled},i,c} \times K_c \quad \text{if } i \in \text{top samples}
$$

$$
N_{\text{final},i,c} = N_{\text{scaled},i,c} \quad \text{if } i \notin \text{top samples}
$$

### Step 4.2: Store final yield

Store $N_{\text{final},i,c}$

---

## PART 5: Calculate Signal Strength per Category

For each category $c$:

### Step 5.1: Sum all signal yields

$$
N_{\text{signal},c} = \sum_{i \in \text{signal samples}} N_{\text{final},i,c}
$$

### Step 5.2: Sum all background yields (including top)

$$
N_{\text{bkg},c} = \sum_{i \in \text{background samples}} N_{\text{final},i,c}
$$

### Step 5.3: Get observed events

$$
N_{\text{obs},c} = N_{\text{data},c}
$$

### Step 5.4: Calculate signal strength

$$
\mu_c = \frac{N_{\text{obs},c} - N_{\text{bkg},c}}{N_{\text{signal},c}}
$$

**Interpretation:**
- $\mu_c = 1$: Standard Model (SM) expectation
- $\mu_c > 1$: Signal excess over SM
- $\mu_c < 1$: Signal deficit compared to SM

### Step 5.5: Calculate statistical uncertainty

$$
\sigma_{\mu,c} = \frac{\sqrt{N_{\text{obs},c}}}{N_{\text{signal},c}}
$$

*(Note: This is a simplified Poisson uncertainty. Use Combine tool for proper treatment.)*

### Step 5.6: Store results

Store $\mu_c$ and $\sigma_{\mu,c}$

---

## PART 6: Calculate Combined Signal Strength

### Step 6.1: Calculate weights for each category

Using inverse-variance weighting:

$$
w_c = \frac{1}{\sigma_{\mu,c}^2}
$$

### Step 6.2: Calculate weighted average

$$
\mu_{\text{combined}} = \frac{\sum_c \mu_c \times w_c}{\sum_c w_c}
$$

### Step 6.3: Calculate combined uncertainty

$$
\sigma_{\text{combined}} = \frac{1}{\sqrt{\sum_c w_c}}
$$

### Step 6.4: Store combined results

Store $\mu_{\text{combined}}$ and $\sigma_{\text{combined}}$

---

## PART 7: Output Results

### Step 7.1-7.6: Save results to CSV files

- `scale_factors.csv`: $\text{SF}_i$ for all MC samples
- `scaled_event_yields.csv`: $N_{\text{scaled},i,c}$ per category
- `k_factors.csv`: $K_c$ per category
- `final_event_yields.csv`: $N_{\text{final},i,c}$ per category
- `signal_strength_by_category.csv`: $\mu_c \pm \sigma_{\mu,c}$
- `combined_signal_strength.csv`: $\mu_{\text{combined}} \pm \sigma_{\text{combined}}$

### Step 7.7: Generate visualization plots

Create plots showing:
- Signal strength by category with combined result
- Event yields comparison (data vs. MC)
- K-factor values
- Sample composition

---

## OUTPUT SUMMARY

| Output | Description | Formula |
|--------|-------------|---------|
| **Scale Factors** | MC normalization to luminosity | $\text{SF}_i = \frac{\sigma_i \times \mathcal{L} \times 1000}{\sum w_{\text{gen},i}}$ |
| **Scaled Yields** | Normalized event counts | $N_{\text{scaled},i,c} = N_{\text{raw},i,c} \times \text{SF}_i$ |
| **K-factors** | Top background data-driven correction | $K_c = \frac{N_{\text{data},c} - N_{\text{other},c}}{N_{\text{top},c}}$ |
| **Final Yields** | K-factor corrected yields | $N_{\text{final},i,c} = N_{\text{scaled},i,c} \times K_c$ (for top) |
| **Signal Strength (per category)** | Observed vs. expected signal | $\mu_c = \frac{N_{\text{obs},c} - N_{\text{bkg},c}}{N_{\text{signal},c}}$ |
| **Combined Signal Strength** | Weighted average across categories | $\mu_{\text{combined}} = \frac{\sum_c \mu_c w_c}{\sum_c w_c}$ where $w_c = \frac{1}{\sigma_{\mu,c}^2}$ |

---

## Key Equations Summary

### 1. MC Normalization
$$
\boxed{\text{SF}_i = \frac{\sigma_i \times \mathcal{L} \times 1000}{\sum w_{\text{gen},i}}}
$$

### 2. K-factor (Top Correction)
$$
\boxed{K_c = \frac{N_{\text{data},c} - N_{\text{other},c}}{N_{\text{top},c}}}
$$

### 3. Signal Strength (POI)
$$
\boxed{\mu_c = \frac{N_{\text{obs},c} - N_{\text{bkg},c}}{N_{\text{signal},c}}}
$$

### 4. Combined Signal Strength
$$
\boxed{\mu_{\text{combined}} = \frac{\sum_c \mu_c / \sigma_{\mu,c}^2}{\sum_c 1 / \sigma_{\mu,c}^2} \pm \frac{1}{\sqrt{\sum_c 1 / \sigma_{\mu,c}^2}}}
$$

---



## Imports

In [1]:
try: 
    import uproot
    import awkward
    import numpy as np
    import os
    import time
    import dask
    from dask import delayed, compute
    from dask.distributed import Client, progress
    from pathlib import Path
    from tqdm import tqdm 
    import pandas as pd
    from datetime import datetime

    print("All imports done!")

except Exception as e:
    print(f"Error: {e}")
    




All imports done!


# Step 1: Find sum of gen weights

In [9]:
# url = "root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/40000/751D5714-5507-6343-818A-5DB7797D6632.root"

# with uproot.open(url) as f:
#     tree = f['Events']
#     for branches in tree.keys():
#         if "weight" in branches.lower():
#             print(f'-{branches}')

These are all the branches in the file with "weight", we are going to use the `genWeight` branch.

In [2]:
BASE_PATH = "/home/cms-jovyan/H-to-WW-NanoAOD-analysis/Datasets/MC_samples"
os.chdir(BASE_PATH)
print(f"The working directory is: \n{os.getcwd()}")

print(f"\nThe files here are")
for file in os.listdir():
    if file.endswith("txt"):
        print(f"-{file}")


The working directory is: 
/home/cms-jovyan/H-to-WW-NanoAOD-analysis/Datasets/MC_samples

The files here are
-VG.txt
-Higgs.txt
-WW.txt
-Fakes.txt
-VZ.txt
-DYtoLL.txt
-ggWW.txt
-Top.txt


In [11]:
def process_sample(txt_file):
    
    sample_name = Path(txt_file).stem
    print(f" Starting: {sample_name}")
    
    
    # Read list of ROOT files
    with open(txt_file) as f:
        root_files = [line.strip() for line in f if line.strip()]

    sum_gen_weight = 0
    n_events = 0
    n_files_processed = 0

    for i, file in enumerate(root_files, 1):
        try:
            with uproot.open(file) as f:
                if "Events" not in f:
                    continue
                
                tree = f["Events"]

                for data in tree.iterate("genWeight", step_size="100MB"):
                    weights = data["genWeight"].to_numpy()
                    sum_gen_weight += np.sum(weights)
                    n_events += len(weights)
            
            n_files_processed += 1
            if i % 50 == 0:  # Print progress every 50 files
                print(f"  {sample_name}: processed {i}/{len(root_files)} files")

        except Exception as e:
            print(f" Error in {sample_name}/{file}: {e}")

    print(f" Completed: {sample_name} ({n_files_processed} files)")
    return sample_name, sum_gen_weight, n_events, n_files_processed


# List of samples
txt_files = [
    "VG.txt", "Higgs.txt", "WW.txt", 63281828Fakes.txt",
    "VZ.txt", "DYtoLL.txt", "ggWW.txt", "Top.txt"
]
# txt_files = ["VG.txt"]

print(f"Processing {len(txt_files)} samples in parallel with 8 workers...\n")

start_time = time.time()


tasks = [dask.delayed(process_sample)(txt_file) for txt_file in txt_files]


with dask.config.set(scheduler='threads', num_workers=8):
    results = dask.compute(*tasks)

total_time = time.time() - start_time

print("\n" + "="*60)
print("RESULTS BY SAMPLE:")
print("="*60)

total_weight = 0
total_events = 0

for sample_name, sum_weight, n_events, n_files in results:
    print(f"{sample_name:15s}: weight={sum_weight:15.2f}, events={n_events:10d}, files={n_files}")
    total_weight += sum_weight
    total_events += n_events


print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

Processing 8 samples in parallel with 8 workers...

 Starting: DYtoLL
 Starting: ggWW
 Starting: Fakes
 Starting: WW
 Starting: VZ
 Starting: Higgs
 Starting: Top
 Starting: VG
 Completed: WW (7 files)
 Completed: Higgs (40 files)
  ggWW: processed 50/112 files
 Completed: VG (32 files)
  VZ: processed 50/73 files
  Top: processed 50/197 files
 Completed: VZ (73 files)
  DYtoLL: processed 50/61 files
  Fakes: processed 50/206 files
  ggWW: processed 100/112 files
 Completed: DYtoLL (61 files)
 Completed: ggWW (112 files)
  Top: processed 100/197 files
  Fakes: processed 100/206 files
  Top: processed 150/197 files
  Fakes: processed 150/206 files
 Completed: Top (197 files)
  Fakes: processed 200/206 files
 Completed: Fakes (206 files)

RESULTS BY SAMPLE:
VG             : weight=  3109819392.00, events=  34915878, files=32
Higgs          : weight=    63281828.00, events=   2946000, files=40
WW             : weight=    32147096.00, events=   2900000, files=7
Fakes          : weight=9740

### Saving the result as a CSV file

In [12]:
df = pd.DataFrame(results, columns=['Sample', 'GenWeight_Sum', 'N_Events', 'N_Files'])


total_rows = pd.DataFrame({
    'Sample': ['TOTAL'],
    'GenWeight_Sum': [df['GenWeight_Sum'].sum()],      
    'N_Events': [df['N_Events'].sum()],                
    'N_Files': [df['N_Files'].sum()]                   
})

df = pd.concat([df, total_rows], ignore_index=True)

# Save to CSV
output_filename = f"/home/cms-jovyan/H-to-WW-NanoAOD-analysis/notebooks/genweight_results.csv"
df.to_csv(output_filename, index=False)

print("\n" + "="*60)
print("RESULTS BY SAMPLE:")
print("="*60)
print(f"saved to csv file: genweight_results.csv")
print("="*60)



RESULTS BY SAMPLE:
saved to csv file: genweight_results.csv


# Step 2: Getting Cross-section of the processes

In [24]:
# The cross-sections for the processes are:

sample_info = {
                "Higgs": {"xsec": 1.0315, "genWeight": 63_281_828.0},
                "DYtoLL": {"xsec": 6189.39, "genWeight": 82_448_512.0},
                "Top": {"xsec": 232.58, "genWeight": 11_433_399_296.0},
                "Fakes": {"xsec": 61_891.05, "genWeight": 9_740_958_564_352.0},
                "VZ": {"xsec": 26.54765, "genWeight": 134_985_184.0},
                "ggWW": {"xsec": 5.7483, "genWeight": 17_662_000.0},
                "WW": {"xsec": 12.178, "genWeight": 32_147_096.0},
                "VG": {"xsec": 464.101, "genWeight": 3_109_819_392.0}
}

Luminosity_pb = 38_250 # in pb^-1

In [25]:
# scale_factors = {}

In [26]:
for sample, info in sample_info.items():
    print(f"-{sample} and {info}")

-Higgs and {'xsec': 1.0315, 'genWeight': 63281828.0}
-DYtoLL and {'xsec': 6189.39, 'genWeight': 82448512.0}
-Top and {'xsec': 232.58, 'genWeight': 11433399296.0}
-Fakes and {'xsec': 61891.05, 'genWeight': 9740958564352.0}
-VZ and {'xsec': 26.54765, 'genWeight': 134985184.0}
-ggWW and {'xsec': 5.7483, 'genWeight': 17662000.0}
-WW and {'xsec': 12.178, 'genWeight': 32147096.0}
-VG and {'xsec': 464.101, 'genWeight': 3109819392.0}


In [38]:
def get_scale_factors(sample_info, Luminosity_pb):
    scale_factors = {}
    for sample, info in sample_info.items():
        sf = (info['xsec']*Luminosity_pb)/info['genWeight']
        scale_factors[sample] = sf
    return scale_factors

In [39]:
sf_dict = get_scale_factors(sample_info, Luminosity_pb)

In [42]:
for sample, sf in sf_dict.items():
    print(f"{sample}: {sf:.9f}")

Higgs: 0.000623479
DYtoLL: 2.871418316
Top: 0.000778087
Fakes: 0.000243029
VZ: 0.007522660
ggWW: 0.012448900
WW: 0.014489909
VG: 0.005708326
